# COCO Image Tiler

Tile larger COCO annotated images into smaller COCO annotated images

This involves cutting up the images and fixing the COCO annotations so that they align with the new images

In [16]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import io
from PIL import Image

import os
print(f"Current OS: {os.name}") 

Current OS: nt


## Utils

In [18]:
# Plot an image from a TF Record with its bounding boxes and labels

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plt_image_with_labels(image, annotations, categories):
    ## Display the image, with bboxes and labels
    # Create figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Display the image
    ax.imshow(image)
    height = annotations["image/height"]
    width = annotations["image/width"]
    # Create Rectangle patches, i.e. bbox add rectangles to the image
    for j in range(len(annotations['image/object/class/label'])):
        x1 = annotations["image/object/bbox/xmin"][j] * width
        y1 = annotations["image/object/bbox/ymin"][j] * height
        x2 = annotations["image/object/bbox/xmax"][j] * width - x1
        y2 = annotations["image/object/bbox/ymax"][j] * height - y1
        
        category_num = annotations["image/object/class/label"][j]-1
        label = categories[category_num]["name"]
        rect = patches.Rectangle((x1, y1), x2, y2, linewidth=1, edgecolor='red', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.text(x1,y1, label, horizontalalignment='left', fontsize=8, color='red')

    plt.show()

In [20]:
# Plot an image from a TF Record with its bounding boxes and labels

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plt_image_with_labels2(image, annotations, categories):
    ## Display the image, with bboxes and labels
    # Create figure and axes
    fig, ax = plt.subplots(figsize=(7, 7))

    # Display the image
    ax.imshow(image)
    height = annotations['height']
    width = annotations['width']
    # Create Rectangle patches, i.e. bbox add rectangles to the image
    for row in annotations['annotations']:
        x1, y1, x2, y2 = row['bbox']
        
        category_num = row['category_id']-1
        label = categories[category_num]["name"]
        rect = patches.Rectangle((x1, y1), x2, y2, linewidth=1, edgecolor='red', facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        ax.text(x1,y1, label, horizontalalignment='left', fontsize=8, color='red')

    plt.show()

## Application

In [4]:
# Define a list of folder paths to be created (if needed) and used later
paths = {
    #'IMAGE_SOURCE_PATH':os.path.join('images'),
    'IMAGE_SOURCE_PATH':os.path.join('E:', os.sep,'Users','Vince','Datasets','NZRC','Gordon','CycloneGitaRawImagery','ML4DR_20210910_01'),
    
    # bounding box annotation
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
}


In [5]:
root_dir = os.path.join('E:', os.sep ,'Users', 'Vince', 'Datasets', 'NZRC', 'Gordon', 'CycloneGitaRawImagery')
tfrecords_dir = os.path.join(root_dir, "ML4DR_20210910_01/tfrecords/val2017")
images_dir = os.path.join(root_dir, "ML4DR_20210910_01")
annotations_dir = os.path.join(root_dir, "ML4DR_20210910_01")
annotation_file = os.path.join(annotations_dir, "ML4DR_20210910_01_coco.json")
paths['IMAGE_PATH'] = images_dir


In [6]:
#Load COCO file information

import json

#Load image refs from COCO file
with open(annotation_file, "r") as f:
    coco_images = json.load(f)["images"]
    
#Load annotations from COCO file
with open(annotation_file, "r") as f:
    coco_annotations = json.load(f)["annotations"]
    
#Load categories from COCO file
with open(annotation_file, "r") as f:
    coco_categories = json.load(f)["categories"]

In [9]:
#Rearrange COCO file info into a form that can be used to create the TF Record file
# this requires the annotation info for each image be associated to the image info
# and it requires a category dictionary to convert from category_id to category_text

#Work through coco-images
# Extract the image attributes
# For each image get a list of annotations
# Add the annotations list to the image

imageInfoList = []
for image_info in coco_images:
    annotationsList = []
    for annotation in coco_annotations:
        if image_info['id'] == annotation['image_id']:
            annotationsList.append(annotation)
    if len(annotationsList)>0:
        image = image_info
        image['annotations'] = annotationsList
        imageInfoList.append(image)
        
categories = {}
labels = []
for cat in coco_categories:
    labels.append({'name':cat['name'], 'id':cat['id']})
    categories[cat['id']] = cat['name']

In [23]:
# What is this for????
#imageInfo = imageInfoList[0]
#tf_example = scan_image_info(imageInfo, paths['IMAGE_SOURCE_PATH'], image_size)
#print('type(tf_example) =',type(tf_example))

In [24]:
print(len(imageInfoList))

49


In [122]:
# fetch the imageInfo and the image

imageInfo = imageInfoList[2]
image_path = paths['IMAGE_SOURCE_PATH']

with tf.io.gfile.GFile(os.path.join(image_path, '{}'.format(imageInfo['file_name'])), 'rb') as fid:
    encoded_jpg = fid.read()
encoded_jpg_io = io.BytesIO(encoded_jpg)
image = Image.open(encoded_jpg_io)

print(image_path, imageInfo['file_name'])
print('type(encoded_jpg) =', type(encoded_jpg))
print('type(encoded_jpg_io) =', type(encoded_jpg_io))
print('type(image) =', type(image))
print('imageInfo:')
#print(imageInfo)

#plt_image_with_labels2(image, imageInfo, coco_categories)

E:\Users\Vince\Datasets\NZRC\Gordon\CycloneGitaRawImagery\ML4DR_20210910_01 Day_1_Images_Flight_1_DSC09232_geotag.JPG
type(encoded_jpg) = <class 'bytes'>
type(encoded_jpg_io) = <class '_io.BytesIO'>
type(image) = <class 'PIL.JpegImagePlugin.JpegImageFile'>
imageInfo:


In [103]:
# Extract bboxes that are within the crop_box
# and adjust their coordinates to fit within the cropped image

def get_bboxes(crop_box, imageInfo):
    adj_bboxes = []
    for a in imageInfo['annotations']:
        bbox_x1, bbox_y1, bbox_x2, bbox_y2 = a['bbox']
        bbox_x2 += bbox_x1
        bbox_y2 += bbox_y1
        crop_x1, crop_y1, crop_x2, crop_y2 = crop_box

        # If top-left bbox corner is inside the crop box
        if crop_x1 < bbox_x1 and crop_y1 < bbox_y1:
            # If bottom-right bbox corner is inside the crop box
            if bbox_x2 < crop_x2 and bbox_y2 < crop_y2:
                adj_bboxes.append([a['bbox'][0]-crop_x1, a['bbox'][1]-crop_y1, a['bbox'][2], a['bbox'][3]])

    return adj_bboxes
    #return imageInfo

In [118]:
# Extract image tiles
import numpy as np
import matplotlib.pyplot as plt

crop_size = (1000, 1000)
crop_overlap = 200
image_size = image.size
print('image_size =', image_size)

img_count = 0
new_imageInfo = imageInfo
new_imageInfoList = []
for x in range(0, image.size[0]-crop_overlap, crop_size[0]-crop_overlap):
    if x >= image_size[0]-crop_size[0]:
        x = image_size[0]-crop_size[0]
    for y in range(0, image.size[1]-crop_overlap, crop_size[1]-crop_overlap):
        img_count += 1
        if y >= image_size[1]-crop_size[1]:
            y = image_size[1]-crop_size[1]
        crop_box = (x, y, x+crop_size[0], y+crop_size[1])

        bboxes = get_bboxes(crop_box, imageInfo)
        #fig, ax = plt.subplots(figsize=(7, 7))
        #ax.imshow(image.crop(crop_box))
        #plt.show()

        #plt_image_with_labels(image_np, item, coco_categories)
        new_imageInfo['id'] = imageInfo['id'] + img_count
        new_imageInfo['width'], new_imageInfo['height'] = crop_size
        #Need a function to insert the img_count into the file name
        # new_imageInfo['file_name'] = imageInfo['file_name'] + '_' + img_count
        # new_imageInfo['flickr_url'] = imageInfo['flickr_url']
        # new_imageInfo['coco_url'] = imageInfo['coco_url']

        new_imageInfoList.append(new_imageInfo)
        
#return new_imageInfo

image_size = (6000, 4000)


In [117]:
img_count

40

In [ ]:
new_imageInfo = imageInfo

In [120]:
new_imageInfo['id'] = NewImageInfo['id'] * img_count
new_imageInfo['width'] = 
new_imageInfo['height'] = 


960

In [121]:
imageInfo

{'id': 24,
 'width': 6000,
 'height': 4000,
 'file_name': 'Day_1_Images_Flight_1_DSC09232_geotag.JPG',
 'license': 0,
 'flickr_url': './Day_1_Images_Flight_1_DSC09232_geotag.JPG',
 'coco_url': './Day_1_Images_Flight_1_DSC09232_geotag.JPG',
 'date_captured': '',
 'annotations': [{'segmentation': [[3631,
     308,
     3779,
     308,
     3779,
     446,
     3631,
     446]],
   'area': 20424,
   'bbox': [3631, 308, 148, 138],
   'iscrowd': 0,
   'id': 3,
   'image_id': 24,
   'category_id': 1},
  {'segmentation': [[3788, 529, 3896, 529, 3896, 638, 3788, 638]],
   'area': 11772,
   'bbox': [3788, 529, 108, 109],
   'iscrowd': 0,
   'id': 4,
   'image_id': 24,
   'category_id': 1},
  {'segmentation': [[3883, 437, 4100, 437, 4100, 655, 3883, 655]],
   'area': 47306,
   'bbox': [3883, 437, 217, 218],
   'iscrowd': 0,
   'id': 5,
   'image_id': 24,
   'category_id': 1},
  {'segmentation': [[4006, 890, 4228, 890, 4228, 1093, 4006, 1093]],
   'area': 45066,
   'bbox': [4006, 890, 222, 203],
